In [1]:
import pandas as pd
import numpy as np

In [ ]:
def read_and_concatenate_csvs(base_filename, start_year, end_year):
    """
    Reads and concatenates CSV files from a range of years.
    
    Parameters:
    - base_filename: The filename pattern without the year (e.g., "dsc_fc_summed_spectra_{}_v01.csv")
    - start_year: The starting year
    - end_year: The ending year
    
    Returns:
    - Concatenated DataFrame
    """
    # List to hold individual DataFrames
    dfs = []
    
    for year in range(start_year, end_year + 1):
        # Format the filename for the current year
        filename = base_filename.format(year)
        
        # Read the CSV into a DataFrame
        df = pd.read_csv(filename, delimiter=',', parse_dates=[0], 
                         infer_datetime_format=True, na_values='0', header=None)
        
        # Append the DataFrame to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list
    concatenated_df = pd.concat(dfs, ignore_index=True)
    
    return concatenated_df

# Use the function to read and concatenate files from 2016 to 2023 (for example)
data = read_and_concatenate_csvs("./BUCKET/Repository/dsc_fc_summed_spectra_{}_v01.csv", 2016, 2023)

In [4]:
data.to_csv("./BUCKET/DSCOVR_MERGED.csv", header=None, index_label=0)

In [24]:
data.head(5)[0].dt.date

0    2016-01-01
1    2016-01-01
2    2016-01-01
3    2016-01-01
4    2016-01-01
Name: 0, dtype: object

In [2]:
# Specify the custom delimiter and header options
delimiter = r'\s+' # Regular expression for one or more whitespace characters
header = None # No header in the CSV file


# Read the CSV file using pandas
kp_data = pd.read_csv("BUCKET/k_indexes/Kp_ap_Ap_SN_F107_since_1932.txt", delimiter=delimiter, header=header, comment="#")


# Rename the columns based on the given format
columns = [
    'Year', 'Month', 'Day', 
    'days', 'days_m', 
    'BSR', 'dB',
    'Kp1', 'Kp2', 'Kp3', 'Kp4', 'Kp5', 'Kp6', 'Kp7', 'Kp8',
    'ap1', 'ap2', 'ap3', 'ap4', 'ap5', 'ap6', 'ap7', 'ap8',
    'Ap', 'SN', 'F10.7obs', 'F10.7adj', 'D'
]

kp_data.columns = columns

In [20]:
kp_data["Date"] = pd.to_datetime(kp_data[["Year", "Month", "Day"]])
kp_data.sample(5)

,Year,Month,Day,days,days_m,BSR,dB,Kp1,Kp2,Kp3,...,ap5,ap6,ap7,ap8,Ap,SN,F10.7obs,F10.7adj,D,Date
25752,2002,7,4,25752,25752.5,2306,4,1.333,0.667,0.667,...,3,2,5,15,5,133,146.3,151.3,2,2002-07-04
17620,1980,3,29,17620,17620.5,2004,26,2.333,2.333,2.667,...,4,7,7,9,8,270,200.5,200.0,2,1980-03-29
513,1933,5,28,513,513.5,1371,10,4.000,2.000,0.667,...,4,4,3,4,7,0,-1.0,-1.0,2,1933-05-28
28797,2010,11,4,28797,28797.5,2418,25,1.667,1.333,0.000,...,5,3,0,3,3,14,79.2,77.8,2,2010-11-04
17833,1980,10,28,17833,17833.5,2012,23,0.000,0.000,0.333,...,2,2,3,3,2,193,180.5,178.1,2,1980-10-28


In [28]:
merged_data = kp_data.merge(data, left_on=kp_data.Date.dt.date, right_on=data[0].dt.date, how="inner")
merged_data.sample(5)

,key_0,Year,Month,Day,days,days_m,BSR,dB,Kp1,Kp2,...,44,45,46,47,48,49,50,51,52,53
1771484,2020-06-21,2020,6,21,32314,32314.5,2549,5,0.667,0.667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
merged_data = merged_data.rename(columns={
    "key_0": "join_date",
    0: "timestamp",
    1: "gse_x",
    2: "gse_y",
    3: "gse_z",
})

In [37]:
merged_data = merged_data.rename(columns={
    i: f"raw_{i}" for i in range(4, 54)
})

In [39]:
merged_data.to_csv("./BUCKET/merged_kp_dscovr.csv", header=True, index=False)

In [52]:
merged_data.to_parquet("./BUCKET/merged_kp_dscovr.parquet", index=False)

In [51]:
merged_data.sample(10000).to_csv("./BUCKET/sample_data_2.csv", header=True, index=False)

In [50]:
merged_data.columns

Index(['join_date', 'Year', 'Month', 'Day', 'days', 'days_m', 'BSR', 'dB',
       'Kp1', 'Kp2', 'Kp3', 'Kp4', 'Kp5', 'Kp6', 'Kp7', 'Kp8', 'ap1', 'ap2',
       'ap3', 'ap4', 'ap5', 'ap6', 'ap7', 'ap8', 'Ap', 'SN', 'F10.7obs',
       'F10.7adj', 'D', 'Date', 'timestamp', 'gse_x', 'gse_y', 'gse_z',
       'raw_4', 'raw_5', 'raw_6', 'raw_7', 'raw_8', 'raw_9', 'raw_10',
       'raw_11', 'raw_12', 'raw_13', 'raw_14', 'raw_15', 'raw_16', 'raw_17',
       'raw_18', 'raw_19', 'raw_20', 'raw_21', 'raw_22', 'raw_23', 'raw_24',
       'raw_25', 'raw_26', 'raw_27', 'raw_28', 'raw_29', 'raw_30', 'raw_31',
       'raw_32', 'raw_33', 'raw_34', 'raw_35', 'raw_36', 'raw_37', 'raw_38',
       'raw_39', 'raw_40', 'raw_41', 'raw_42', 'raw_43', 'raw_44', 'raw_45',
       'raw_46', 'raw_47', 'raw_48', 'raw_49', 'raw_50', 'raw_51', 'raw_52',
       'raw_53'],
      dtype='object')

In [1]:
import pandas as pd
import numpy as np
all_data = pd.read_csv('./BUCKET/DSCOVR_KP_PER_INTERVAL.csv')

In [3]:
all_data.to_parquet("./BUCKET/DSCOVR_KP_PER_INTERVAL.parquet", index=False)